In [1]:
import nltk
import numpy as np
import string
from itertools import combinations
from nltk.corpus import brown

In [2]:
from nltk.corpus import stopwords
stop = stopwords.words('english') + [i for i in string.punctuation]

In [17]:
#Sentence Extraction
#similarity function
def sent_sim(s1, s2):
    common = float(sum([word in s2 for word in s1]))
    if common == 0:
        return 0.0
    else:
        return common/ (np.log(len(s1)) + np.log(len(s2)))

In [ ]:
#Normal TextRank

In [33]:
sentences = brown.sents('ca19')
G = np.diag([1.0]*len(sentences))
tagged_sentences = brown.tagged_sents('ca19')
scores = [1.0]*len(sentences)
filtered_sentences = [[word[0].lower() for word in sentence 
                       if word[0] not in stop and word[1] in
                       ['NN', 'JJ', 'VB', 'NP', 'NNS', 'RB', 'VBN', 'VBG'] 
                       and len(word[0]) > 2] for sentence in tagged_sentences]
#populate graph
for i,j in combinations(range(len(filtered_sentences)), 2):
    G[i][j] = sent_sim(filtered_sentences[i], filtered_sentences[j])

In [34]:
def score_sentence(G, v, d, scores):
    return (1.0 - d) + d*sum([float(scores[i])*G[v][i]/sum(G.T[i]) for i in xrange(len(G[v]))])

converged = False
while not converged:
    converged = True
    for node in xrange(len(sentences)):
        old_score = scores[node]
        scores[node] = score_sentence(G, node, 0.85, scores)
        if abs(scores[node] - old_score) > 0.0001:
            converged = False
            
summary_sents = sorted([index for (score, index) in sorted([(scores[i], i) for i in xrange(len(scores))], reverse=True)[:5]])            


In [36]:
for sent in summary_sents:
    print ' '.join(sentences[sent])

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
The reduction in expenses will affect employees in the thirteen states in which the B. & O. operates .
A thug struck a cab driver in the face with a pistol last night after robbing him of $18 at Franklin and Mount Streets .
A baby was burned to death and two other children were seriously injured last night in a fire which damaged their one-room Anne Arundel county home .
His sister and brother , Marie Louise , 3 , and John Raymond , Jr. 22 months , were admitted to the hospital .


In [ ]:
#Modified TextRank

In [55]:
sentences = brown.sents('ca19')
G = np.diag([1.0]*len(sentences))
tagged_sentences = brown.tagged_sents('ca19')
scores = [1.0]*len(sentences)
summary = []
summary_words = set()
while len(summary) < 5:
    filtered_sentences = [[word[0].lower() for word in sentence 
                           if word[0] not in stop and word[1] in
                           ['NN', 'JJ', 'VB', 'NP', 'NNS', 'RB', 'VBN', 'VBG'] and word[0] not in summary_words
                           and len(word[0]) > 2] for sentence in tagged_sentences]
    #populate graph
    for i,j in combinations(range(len(filtered_sentences)), 2):
        G[i][j] = sent_sim(filtered_sentences[i], filtered_sentences[j])
        
    converged = False
    while not converged:
        converged = True
        for node in xrange(len(sentences)):
            old_score = scores[node]
            scores[node] = score_sentence(G, node, 0.85, scores)
            if abs(scores[node] - old_score) > 0.0001:
                converged = False
    new_sent = sorted([(scores[i], i) for i in xrange(len(scores))], reverse=True)[0][1]
    summary.append(new_sent)
    summary_words = summary_words.union(sentences[new_sent])
    


In [58]:
for sent in sorted(summary):
    print ' '.join(sentences[sent])

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
The reduction in expenses will affect employees in the thirteen states in which the B. & O. operates .
A thug struck a cab driver in the face with a pistol last night after robbing him of $18 at Franklin and Mount Streets .
The driver told police he followed as the Negro man got out of the cab with his money .
A baby was burned to death and two other children were seriously injured last night in a fire which damaged their one-room Anne Arundel county home .


In [60]:
from nltk.corpus import wordnet as wn

In [63]:
wn.synset("dog")

ValueError: need more than 1 value to unpack

In [ ]:
def textrank(D, k):
    from nltk.corpus import stopwords
